In [12]:
from PIL import Image, ImageTk
import numpy as np
from pandas import DataFrame, Series
import serial
from threading import Thread
import os

In [14]:
from tkinter.ttk import Button
from tkinter import Tk, END, BOTTOM, LEFT, RIGHT, Label, Frame
from tkinter import filedialog

class UpdateResults(Thread):
    def __init__(self, app):
        super().__init__()

        self.app = app
        self.s = app.s
        self.show_message = app.show_message
        self.highlight_box = app.highlight_box

    def run(self):
        sensor_readings, i = np.zeros((len(app.guestures))), 0
        while self.app.listen:
            data = self.s.readline().decode().strip().split()
            if data != []:
                sensor_readings[i] = float(data[1])
                i += 1
            else:
                index = sensor_readings.argmax()
                guesture = app.guestures[index]
                self.show_message(guesture)
                self.highlight_box(index)
                sensor_readings, i = np.zeros((len(app.guestures))), 0

class CollectData(Thread):
    def __init__(self, app):
        super().__init__()

        self.readLine = app.readLine
        self.show_message = app.show_message
        self.guestures = app.guestures
        self.data_export_path = app.data_export_path
        self.highlight_box = app.highlight_box
        self.export_duration = app.export_duration

    def run(self):
        sensor_step = self.readLine()
        self.show_message("WAITING FOR ARDUINO")
        while sensor_step[0] != "# 1": #wait till it is a begining
            sensor_step = self.readLine()

        columns = ["aX", "aY", "aZ", "gX", "gY", "gZ"]
        for i, guesture in enumerate(self.guestures):
            df = DataFrame(columns=columns)

            self.highlight_box(i, "gray")
            while sensor_step[0][0] == "#":
                if sensor_step[0][1] == " ":
                    self.show_message(f"READY FOR {guesture} in {sensor_step[0][2]}")
                sensor_step = self.readLine()
                
            self.highlight_box(i, "yellow")
            self.show_message(f"START {guesture} ({self.export_duration}s)")
            sensor_step_float = tuple(map(float, sensor_step))
            df = df.append(Series(sensor_step_float, index=columns),ignore_index=True)
            while True:
                sensor_step = self.readLine()
                if sensor_step[0][0] == "#": break
                sensor_step_float = tuple(map(float, sensor_step))
                df = df.append(Series(sensor_step_float, index=columns),ignore_index=True)

            self.show_message(f"STOP {guesture}")
            df.to_csv(f"{self.data_export_path}/{guesture}.csv", index=False)
            
        self.highlight_box(i, "white")
        self.show_message(f"DONE!")

class App(Tk):  
    def __init__(self, s, data_export_path, export_duration):
        super().__init__()
        framePaddings = {'padx': 10, 'pady': 10}
        paddings = {'padx': 5, 'pady': 5}
        box_size = 200
        img_size = 200
        entry_font = {'font': ('Helvetica', 1)}
        classFrame = Frame(self)
        classFrame.pack(**framePaddings)
        guestures = [
            "rub_palms",
            "palm_over_dorsum_left",
            "palm_over_dorsum_right",
            "palm_to_palm_fingers",
            "fingers_interlocked",
            "thumb_rub_left",
            "thumb_rub_right",
            "palms_with_fingers_left",
            "palms_with_fingers_right",
            "misc",
        ]
        boxes = []
        for i in range(len(guestures)//2):
            guesture1, guesture2 = guestures[i*2], guestures[i*2+1]

            image1 = Image.open(f"./img/{guesture1}.jpg").resize((img_size, img_size))
            image1 = ImageTk.PhotoImage(image1)
            box1 = Label(classFrame, image=image1, text=guesture1, font=('Times 12'), width=box_size, height=box_size, compound='center')
            box1.image=image1
            box1.grid(column=i+1, row=1, **paddings)
            
            image2 = Image.open(f"./img/{guesture2}.jpg").resize((img_size, img_size))
            image2 = ImageTk.PhotoImage(image2)
            box2 = Label(classFrame, text=guesture2, image=image2, font=('Times 12'), width=box_size, height=box_size, compound='center')
            box2.image=image2
            box2.grid(column=i+1, row=2, **paddings)
            boxes.extend([box1, box2])

        notificationFrame = Frame(self)
        notificationFrame.pack(**paddings)
        notificationLabel = Label(notificationFrame)
        notificationLabel.pack(**paddings)

        resultsFrame = Frame(self)
        resultsFrame.pack(**framePaddings)

        buttonFrame = Frame(self)
        buttonFrame.pack(side = BOTTOM, **framePaddings)
        collectDataBtn = Button(buttonFrame, text="Collect data", command=self.collect_data)
        collectDataBtn.pack(side = LEFT)
        listenBtn = Button(buttonFrame, text="Listen", command=self.listen)
        listenBtn.pack(side = LEFT)
        closeBtn = Button(buttonFrame, text="Close", command=self.destroy)
        closeBtn.pack(side = RIGHT)

        self.boxes = boxes
        self.listen = False
        self.collect_data = False
        self.s = s
        self.export_duration = export_duration
        self.guestures = guestures
        self.notificationLabel = notificationLabel
        self.data_export_path = data_export_path

    def readLine(self):
        sensor_step = self.s.readline().decode().strip().split(",")
        return sensor_step
    
    def collect_data(self):
        self.collect_data = not self.collect_data
        collect_data_thread = CollectData(self)
        collect_data_thread.start()

    def listen(self):
        self.listen = not self.listen
        update_results_thread = UpdateResults(self)
        update_results_thread.start()

    def highlight_box(self, index, color="yellow"):
        for image_box in self.boxes:
            image_box.config(background="white")
        selected_image_box = self.boxes[index]
        selected_image_box.config(background=color)

    def show_error(self, message):
        self.notificationLabel.configure(text=message, background="pink")

    def show_message(self, message):
        self.notificationLabel.configure(text=message, background="white")

    def destroy(self):
        self.s.close()
        super().destroy()

s = serial.Serial('COM3', 9600, timeout=1)
app = App(s, "../DataCapture/data/data_60s", 60)
app.mainloop()

In [ ]:
s = serial.Serial('COM3', 9600, timeout=1)

In [ ]:
while True:
    data = s.readline().decode().strip().split()
    if data != []:
        print(data[1], end=" ")
    else:
        print()

In [12]:
s.close()